<a href="https://colab.research.google.com/github/YoshitakaNaraoka/gColabNotebooks/blob/main/%E3%82%B1%E3%83%A2%E3%82%A4%E3%83%B3%E3%83%95%E3%82%A9%E7%A0%82%E5%A0%B4%E5%85%BC%E5%82%99%E5%BF%98%E9%8C%B2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!wget -c https://repo.continuum.io/miniconda/Miniconda3-latest-Linux-x86_64.sh
!chmod +x Miniconda3-latest-Linux-x86_64.sh
!bash ./Miniconda3-latest-Linux-x86_64.sh -b -f -p /usr/local
%conda install -q -y -c rdkit rdkit python=latest
import sys
sys.path.append('/usr/local/lib/python/site-packages/')

In [ ]:
#pip install psi4 python=3.7 -c psi4 -c conda-forge

# Psi4のインストール
%conda install -y psi4 python=latest -c psi4
# モジュール検索パスの追加
import sys
sys.path.append("/usr/local/lib/python/site-packages/")

In [ ]:
# セル実行時間の測定
%%time

# rdkitをインストール
!curl -Lo rdkit_installer.py https://git.io/fxiPZ
import rdkit_installer

# rdkit内のChemモジュールを読み込む(ここをいったん動かさないと以下のChem.~が動いてくれない)
from rdkit import Chem

from rdkit.Chem.Draw import IPythonConsole
from IPython.display import display

# gzipモジュールを読み込む
import gzip

# 分子の標準化を行うためのモジュールを読み込む
from rdkit.Chem import MolStandardize


In [ ]:
# 3D表示のためのpy3Dmolをインストール
%pip install py3Dmol

# mordredのインストール
%pip install mordred

In [ ]:
# MolFromSmilesで生成した分子を構造式表示するにはdisplayコマンド
benzene = Chem.MolFromSmiles("c1ccccc1")
display(benzene)

# 読み込んだファイル内の分子は配列形式で表示可能
EPc_mols[2]

In [ ]:
EPc_test = EPc_mols[2]
names=list(EPc_test.GetPropNames())
print(names)

In [ ]:
#標準化
normalizer =MolStandardize.normalize.Normalizer()
test_mol_norm = normalizer.normalize(EPc_test)

#一番大きいサイズのフラグメントのみ残す（ここで脱塩されるみたい）
lfc = MolStandardize.fragment.LargestFragmentChooser()
test_mol_desalt = lfc.choose(test_mol_norm)

#電荷の中和
uc = MolStandardize.charge.Uncharger()
test_mol_neu = uc.uncharge(test_mol_desalt)

In [ ]:
test_mol_desalt

In [ ]:
#セルの処理の時間測定
%%time

#前処理を実行して新しいMOlオブジェクトのリストを作る
#空のリストを作成
processed_EPc_mols = []

#ループを回せ!!!
for i in range(len(EPc_mols)):
  mol = EPc_mols[i]

  #構造の標準化
  normalizer =MolStandardize.normalize.Normalizer()
  mol_norm = normalizer.normalize(mol)

  #一番大きいサイズのフラグメントのみ残す
  lfc = MolStandardize.fragment.LargestFragmentChooser()
  mol_desalt = lfc.choose(mol_norm)

  #電荷の中和
  uc = MolStandardize.charge.Uncharger()
  mol_neu = uc.uncharge(mol_desalt)

  #新しいリストに追加
  processed_EPc_mols.append(mol_neu)

In [ ]:
#構造とidnumberのみを残したsdfファイルを作成
#SDWriterを使用する
writer = Chem.SDWriter('Enamine_Premium_collection_processed.sdf')

#プロパティの設定
#右だとうまくいきません。プロパティはリストで渡す必要があります。→ writer.SetProps('idnumber')
writer.SetProps(['idnumber'])

#ループを回せ！！
for mol in processed_EPc_mols:
  writer.write(mol)

#そっ閉じ
writer.close()


In [ ]:
#大きいので圧縮
#colaboratoryでは"!"を先につけるらしい
!gzip -c Enamine_Premium_collection_processed.sdf > Enamine_Premium_collection_processed.sdf.gz

#Googleドライブへ出力
upload_file = drive.CreateFile()
upload_file.SetContentFile('Enamine_Premium_collection_processed.sdf.gz')
upload_file.Upload()

In [ ]:
crown = Chem.MolFromSmiles('C1COCCOCCOCCOCCOCCO1')
display(crown)

In [ ]:
# CoordGen ライブラリでは大員環化合物を見やすく描画できる
from rdkit.Chem import rdCoordGen
Chem.rdCoordGen.AddCoords(crown)
display(crown)

In [ ]:
alanine = Chem.MolFromSmiles('CC(C(=O)O)N')
phenylalanine = Chem.MolFromSmiles('c1ccccc1CC(C(=O)O)N')
display(alanine,phenylalanine)

In [ ]:
# TPSA(極性表面積)-細胞膜透過性の指標
from rdkit import rdBase, Chem
from rdkit.Chem import Draw, Descriptors
### 分子の用意
alanine = Chem.MolFromSmiles('CC(C(=O)O)N')
phenylalanine = Chem.MolFromSmiles('c1ccccc1CC(C(=O)O)N')
mol_list = [alanine,phenylalanine]
writer=Chem.SDWriter('molecules.sdf')
for mol in mol_list:
	writer.write(mol)
writer.close()
suppl = Chem.SDMolSupplier('molecules.sdf')
mols = [x for x in suppl if x is not None]
len(mols) # 2
### TPSAの計算
declist = Descriptors.descList
calc = {}
for (i,j) in declist:
    calc[i] = j
tpsa = [calc['TPSA'](i) for i in mols]
### 描画
display(mol_list[0])
display(mol_list[1])
print(tpsa)
